### Using all 8 stations with a P record from 2005-2016: 
### Fisera Ridge, Upper Clearing, Hay Meadow
### Kananaskis, Bow Valley, Kananaskis Pocaterra, Banff CS, Wildcat Hills
### Remove dry days for all thresholds considered: P=0mm, P<0.1mm, P<=0.1mm, P<0.2mm, P<1mm

In [1]:
import numpy as np
import pandas as pd
import math
from scipy import stats
from sklearn.linear_model import LinearRegression
import scipy
# import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

### Read in precip data for all stations

### Fisera Ridge 2325 m, Upper Clearing 1845 m, Hay Meadow 1436 m

In [2]:
# Use pd.read_csv to read csv file
# Fisera Ridge
FRP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Version 02\Meteorology\Recent_modelling_data\time_series_inputs\FRdailyprecip.csv", 
                            header=None, names= ["date", "FR_p_mm"])
FRP_df.set_index("date", inplace=True)
# Upper Clearing
UCP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Version 02\Meteorology\Recent_modelling_data\time_series_inputs\UCdailyprecip.csv", 
                            header=None, names= ["date", "UC_p_mm"])
UCP_df.set_index("date", inplace=True)
# Hay Meadow
HMP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Version 02\Meteorology\Recent_modelling_data\time_series_inputs\HMdailyprecip.csv", 
                            header=None, names= ["date", "HM_p_mm"])
HMP_df.set_index("date", inplace=True)
# threshold = 0 mm
FRP0mm_df = FRP_df[FRP_df!=0].dropna()
UCP0mm_df = UCP_df[UCP_df!=0].dropna()
HMP0mm_df = HMP_df[HMP_df!=0].dropna()
# threshold < 0.1 mm
FRPpt1mm_df = FRP_df.drop(FRP_df[FRP_df.FR_p_mm < 0.1].index)
UCPpt1mm_df = UCP_df.drop(UCP_df[UCP_df.UC_p_mm < 0.1].index)
HMPpt1mm_df = HMP_df.drop(HMP_df[HMP_df.HM_p_mm < 0.1].index)
# threshold <= 0.1 mm
FRPleqpt1mm_df = FRP_df.drop(FRP_df[FRP_df.FR_p_mm <= 0.1].index)
UCPleqpt1mm_df = UCP_df.drop(UCP_df[UCP_df.UC_p_mm <= 0.1].index)
HMPleqpt1mm_df = HMP_df.drop(HMP_df[HMP_df.HM_p_mm <= 0.1].index)
# threshold < 0.2 mm
FRPpt2mm_df = FRP_df.drop(FRP_df[FRP_df.FR_p_mm < 0.2].index)
UCPpt2mm_df = UCP_df.drop(UCP_df[UCP_df.UC_p_mm < 0.2].index)
HMPpt2mm_df = HMP_df.drop(HMP_df[HMP_df.HM_p_mm < 0.2].index)
# threshold < 1.0 mm
FRP1mm_df = FRP_df.drop(FRP_df[FRP_df.FR_p_mm < 1.0].index)
UCP1mm_df = UCP_df.drop(UCP_df[UCP_df.UC_p_mm < 1.0].index)
HMP1mm_df = HMP_df.drop(HMP_df[HMP_df.HM_p_mm < 1.0].index)

### Kananaskis 1391.1 m

In [3]:
# Use pd.read_csv to read csv file
KP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Other_stations_within_50km\Kananaskis\en_climate_daily_AB_3053600_2005-2016_P1D.csv", 
                               header=[0])
KP_df.rename(columns={'Date/Time': 'date', 'Total Precip (mm)': 'K_p_mm'}, inplace=True) # rename date column
KP_df.set_index("date", inplace=True)
# change datetime format to YMD
KP_df.index = pd.to_datetime(KP_df.index, format = '%d/%m/%Y').strftime('%Y-%m-%d')
# drop unwanted columns
KP_df.drop(["Longitude (x)", "Latitude (y)", "Station Name", "Climate ID", "Year", "Month",
                      "Day", "Data Quality", "Max Temp (°C)", "Max Temp Flag", "Min Temp (°C)", "Min Temp Flag",
                      "Mean Temp (°C)", "Mean Temp Flag", "Heat Deg Days (°C)", "Heat Deg Days Flag", "Cool Deg Days (°C)",
                       "Cool Deg Days Flag", "Total Rain (mm)", "Total Rain Flag", "Total Snow (cm)", "Total Snow Flag",
                       "Total Precip Flag", "Snow on Grnd (cm)", "Snow on Grnd Flag", "Dir of Max Gust (10s deg)",
                       "Dir of Max Gust Flag", "Spd of Max Gust (km/h)", "Spd of Max Gust Flag"], axis=1, inplace=True)
# remove NaN values
KP_df["K_p_mm"] = KP_df["K_p_mm"].fillna(0)
# drop 1st Jan to 30th Sep 2005 (MCRB inputs start at 1st Oct 2005)
KP_df.drop(KP_df.loc["2005-01-01":"2005-09-30"].index, inplace=True)
# drop 2nd Oct to 31st Dec 2016 (MCRB inputs end at 1st Oct 2016)
KP_df.drop(KP_df.loc["2016-10-02":"2016-12-31"].index, inplace=True)
# threshold = 0 mm
KP0mm_df = KP_df[KP_df!=0].dropna()
# threshold < 0.1 mm
KPpt1mm_df = KP_df.drop(KP_df[KP_df.K_p_mm < 0.1].index)
# threshold <= 0.1 mm
KPleqpt1mm_df = KP_df.drop(KP_df[KP_df.K_p_mm <= 0.1].index)
# threshold < 0.2 mm
KPpt2mm_df = KP_df.drop(KP_df[KP_df.K_p_mm < 0.2].index)
# threshold < 1.0 mm
KP1mm_df = KP_df.drop(KP_df[KP_df.K_p_mm < 1.0].index)

### Bow Valley 1297.5 m

In [4]:
# Use pd.read_csv to read csv file
BVP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Other_stations_within_50km\Bow_Valley\en_climate_daily_AB_3050778_2005-2016_P1D.csv", 
                               header=[0])
BVP_df.rename(columns={'Date/Time': 'date', 'Total Precip (mm)': 'BV_p_mm'}, inplace=True) # rename date column
BVP_df.set_index("date", inplace=True)
# change datetime format to YMD
BVP_df.index = pd.to_datetime(BVP_df.index, format = '%d/%m/%Y').strftime('%Y-%m-%d')
# drop unwanted columns
BVP_df.drop(["Longitude (x)", "Latitude (y)", "Station Name", "Climate ID", "Year", "Month",
                      "Day", "Data Quality", "Max Temp (°C)", "Max Temp Flag", "Min Temp (°C)", "Min Temp Flag",
                      "Mean Temp (°C)", "Mean Temp Flag", "Heat Deg Days (°C)", "Heat Deg Days Flag", "Cool Deg Days (°C)",
                       "Cool Deg Days Flag", "Total Rain (mm)", "Total Rain Flag", "Total Snow (cm)", "Total Snow Flag",
                       "Total Precip Flag", "Snow on Grnd (cm)", "Snow on Grnd Flag", "Dir of Max Gust (10s deg)",
                       "Dir of Max Gust Flag", "Spd of Max Gust (km/h)", "Spd of Max Gust Flag"], axis=1, inplace=True)
# remove NaN values
BVP_df["BV_p_mm"] = BVP_df["BV_p_mm"].fillna(0)
# drop 1st Jan to 30th Sep 2005 (MCRB inputs start at 1st Oct 2005)
BVP_df.drop(BVP_df.loc["2005-01-01":"2005-09-30"].index, inplace=True)
# need to drop 2nd Oct to 31st Dec 2016 (MCRB inputs end at 1st Oct 2016)
BVP_df.drop(BVP_df.loc["2016-10-02":"2016-12-31"].index, inplace=True)
# threshold = 0 mm
BVP0mm_df = BVP_df[BVP_df!=0].dropna()
# threshold < 0.1 mm
BVPpt1mm_df = BVP_df.drop(BVP_df[BVP_df.BV_p_mm < 0.1].index)
# threshold <= 0.1 mm
BVPleqpt1mm_df = BVP_df.drop(BVP_df[BVP_df.BV_p_mm <= 0.1].index)
# threshold < 0.2 mm
BVPpt2mm_df = BVP_df.drop(BVP_df[BVP_df.BV_p_mm < 0.2].index)
# threshold < 1.0 mm
BVP1mm_df = BVP_df.drop(BVP_df[BVP_df.BV_p_mm < 1.0].index)

### Kananaskis Pocaterra 1610.0 m

In [5]:
# Use pd.read_csv to read csv file
KPP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Other_stations_within_50km\Kananaskis_Pocaterra\en_climate_daily_AB_3053604_2005-2016_P1D.csv", 
                               header=[0])
KPP_df.rename(columns={'Date/Time': 'date', 'Total Precip (mm)': 'KP_p_mm'}, inplace=True) # rename date column
KPP_df.set_index("date", inplace=True)
# change datetime format to YMD
KPP_df.index = pd.to_datetime(KPP_df.index, format = '%d/%m/%Y').strftime('%Y-%m-%d')
# drop unwanted columns
KPP_df.drop(["Longitude (x)", "Latitude (y)", "Station Name", "Climate ID", "Year", "Month",
                      "Day", "Data Quality", "Max Temp (°C)", "Max Temp Flag", "Min Temp (°C)", "Min Temp Flag",
                      "Mean Temp (°C)", "Mean Temp Flag", "Heat Deg Days (°C)", "Heat Deg Days Flag", "Cool Deg Days (°C)",
                       "Cool Deg Days Flag", "Total Rain (mm)", "Total Rain Flag", "Total Snow (cm)", "Total Snow Flag",
                       "Total Precip Flag", "Snow on Grnd (cm)", "Snow on Grnd Flag", "Dir of Max Gust (10s deg)",
                       "Dir of Max Gust Flag", "Spd of Max Gust (km/h)", "Spd of Max Gust Flag"], axis=1, inplace=True)
# remove NaN values
KPP_df["KP_p_mm"] = KPP_df["KP_p_mm"].fillna(0)
# drop 1st Jan to 30th Sep 2005 (MCRB inputs start at 1st Oct 2005)
KPP_df.drop(KPP_df.loc["2005-01-01":"2005-09-30"].index, inplace=True)
# need to drop 2nd Oct to 31st Dec 2016 (MCRB inputs end at 1st Oct 2016)
KPP_df.drop(KPP_df.loc["2016-10-02":"2016-12-31"].index, inplace=True)
# threshold = 0 mm
KPP0mm_df = KPP_df[KPP_df!=0].dropna()
# threshold < 0.1 mm
KPPpt1mm_df = KPP_df.drop(KPP_df[KPP_df.KP_p_mm < 0.1].index)
# threshold <= 0.1 mm
KPPleqpt1mm_df = KPP_df.drop(KPP_df[KPP_df.KP_p_mm <= 0.1].index)
# threshold < 0.2 mm
KPPpt2mm_df = KPP_df.drop(KPP_df[KPP_df.KP_p_mm < 0.2].index)
# threshold < 1.0 mm
KPP1mm_df = KPP_df.drop(KPP_df[KPP_df.KP_p_mm < 1.0].index)

### Banff CS 1396.9 m

In [6]:
# Use pd.read_csv to read csv file
BCSP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Other_stations_within_50km\Banff_CS\en_climate_daily_AB_3050519_2005-2016_P1D.csv", 
                               header=[0])
BCSP_df.rename(columns={'Date/Time': 'date', 'Total Precip (mm)': 'BCS_p_mm'}, inplace=True) # rename date column
BCSP_df.set_index("date", inplace=True)
# change datetime format to YMD
BCSP_df.index = pd.to_datetime(BCSP_df.index, format = '%d/%m/%Y').strftime('%Y-%m-%d')
# drop unwanted columns
BCSP_df.drop(["Longitude (x)", "Latitude (y)", "Station Name", "Climate ID", "Year", "Month",
                      "Day", "Data Quality", "Max Temp (°C)", "Max Temp Flag", "Min Temp (°C)", "Min Temp Flag",
                      "Mean Temp (°C)", "Mean Temp Flag", "Heat Deg Days (°C)", "Heat Deg Days Flag", "Cool Deg Days (°C)",
                       "Cool Deg Days Flag", "Total Rain (mm)", "Total Rain Flag", "Total Snow (cm)", "Total Snow Flag",
                       "Total Precip Flag", "Snow on Grnd (cm)", "Snow on Grnd Flag", "Dir of Max Gust (10s deg)",
                       "Dir of Max Gust Flag", "Spd of Max Gust (km/h)", "Spd of Max Gust Flag"], axis=1, inplace=True)
# remove NaN values
BCSP_df["BCS_p_mm"] = BCSP_df["BCS_p_mm"].fillna(0)
# drop 1st Jan to 30th Sep 2005 (MCRB inputs start at 1st Oct 2005)
BCSP_df.drop(BCSP_df.loc["2005-01-01":"2005-09-30"].index, inplace=True)
# drop 2nd Oct to 31st Dec 2016 (MCRB inputs end at 1st Oct 2016)
BCSP_df.drop(BCSP_df.loc["2016-10-02":"2016-12-31"].index, inplace=True)
# threshold = 0 mm
BCSP0mm_df = BCSP_df[BCSP_df!=0].dropna()
# threshold < 0.1 mm
BCSPpt1mm_df = BCSP_df.drop(BCSP_df[BCSP_df.BCS_p_mm < 0.1].index)
# threshold <= 0.1 mm
BCSPleqpt1mm_df = BCSP_df.drop(BCSP_df[BCSP_df.BCS_p_mm <= 0.1].index)
# threshold < 0.2 mm
BCSPpt2mm_df = BCSP_df.drop(BCSP_df[BCSP_df.BCS_p_mm < 0.2].index)
# threshold < 1.0 mm
BCSP1mm_df = BCSP_df.drop(BCSP_df[BCSP_df.BCS_p_mm < 1.0].index)

### Wildcat Hills 1268.0 m

In [7]:
# Use pd.read_csv to read csv file
WHP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Hydrological_Modelling\Marmot_Creek_Research_Basin\Other_stations_within_50km\Wild_Cat_Hills\en_climate_daily_AB_3037550_2005-2016_P1D.csv", 
                               header=[0])
WHP_df.rename(columns={'Date/Time':'date', 'Total Precip (mm)':'WH_p_mm'}, inplace=True) # rename date column
WHP_df.set_index("date", inplace=True)
# change datetime format to YMD
WHP_df.index = pd.to_datetime(WHP_df.index, format = '%d/%m/%Y').strftime('%Y-%m-%d')
# drop unwanted columns
WHP_df.drop(["Longitude (x)", "Latitude (y)", "Station Name", "Climate ID", "Year", "Month",
                      "Day", "Data Quality", "Max Temp (°C)", "Max Temp Flag", "Min Temp (°C)", "Min Temp Flag",
                      "Mean Temp (°C)", "Mean Temp Flag", "Heat Deg Days (°C)", "Heat Deg Days Flag", "Cool Deg Days (°C)",
                       "Cool Deg Days Flag", "Total Rain (mm)", "Total Rain Flag", "Total Snow (cm)", "Total Snow Flag",
                       "Total Precip Flag", "Snow on Grnd (cm)", "Snow on Grnd Flag", "Dir of Max Gust (10s deg)",
                       "Dir of Max Gust Flag", "Spd of Max Gust (km/h)", "Spd of Max Gust Flag"], axis=1, inplace=True)
# remove NaN values
WHP_df["WH_p_mm"] = WHP_df["WH_p_mm"].fillna(0)
# drop 1st Jan to 30th Sep 2005 (MCRB inputs start at 1st Oct 2005)
WHP_df.drop(WHP_df.loc["2005-01-01":"2005-09-30"].index, inplace=True)
# drop 2nd Oct to 31st Dec 2016 (MCRB inputs end at 1st Oct 2016)
WHP_df.drop(WHP_df.loc["2016-10-02":"2016-12-31"].index, inplace=True)
# threshold = 0 mm
WHP0mm_df = WHP_df[WHP_df!=0].dropna()
# threshold < 0.1 mm
WHPpt1mm_df = WHP_df.drop(WHP_df[WHP_df.WH_p_mm < 0.1].index)
# threshold <= 0.1 mm
WHPleqpt1mm_df = WHP_df.drop(WHP_df[WHP_df.WH_p_mm <= 0.1].index)
# threshold < 0.2 mm
WHPpt2mm_df = WHP_df.drop(WHP_df[WHP_df.WH_p_mm < 0.2].index)
# threshold < 1.0 mm
WHP1mm_df = WHP_df.drop(WHP_df[WHP_df.WH_p_mm < 1.0].index)

### Find the number of wet days for each threshold

In [8]:
FR0mm_wetdays = len(FRP0mm_df)
FRpt1mm_wetdays = len(FRPpt1mm_df)
FRleqpt1mm_wetdays = len(FRPleqpt1mm_df)
FRpt2mm_wetdays = len(FRPpt2mm_df)
FR1mm_wetdays = len(FRP1mm_df)

UC0mm_wetdays = len(UCP0mm_df)
UCpt1mm_wetdays = len(UCPpt1mm_df)
UCleqpt1mm_wetdays = len(UCPleqpt1mm_df)
UCpt2mm_wetdays = len(UCPpt2mm_df)
UC1mm_wetdays = len(UCP1mm_df)

KP0mm_wetdays = len(KPP0mm_df)
KPpt1mm_wetdays = len(KPPpt1mm_df)
KPleqpt1mm_wetdays = len(KPPleqpt1mm_df)
KPpt2mm_wetdays = len(KPPpt2mm_df)
KP1mm_wetdays = len(KPP1mm_df)

HM0mm_wetdays = len(HMP0mm_df)
HMpt1mm_wetdays = len(HMPpt1mm_df)
HMleqpt1mm_wetdays = len(HMPleqpt1mm_df)
HMpt2mm_wetdays = len(HMPpt2mm_df)
HM1mm_wetdays = len(HMP1mm_df)

BCS0mm_wetdays = len(BCSP0mm_df)
BCSpt1mm_wetdays = len(BCSPpt1mm_df)
BCSleqpt1mm_wetdays = len(BCSPleqpt1mm_df)
BCSpt2mm_wetdays = len(BCSPpt2mm_df)
BCS1mm_wetdays = len(BCSP1mm_df)

K0mm_wetdays = len(KP0mm_df)
Kpt1mm_wetdays = len(KPpt1mm_df)
Kleqpt1mm_wetdays = len(KPleqpt1mm_df)
Kpt2mm_wetdays = len(KPpt2mm_df)
K1mm_wetdays = len(KP1mm_df)

BV0mm_wetdays = len(BVP0mm_df)
BVpt1mm_wetdays = len(BVPpt1mm_df)
BVleqpt1mm_wetdays = len(BVPleqpt1mm_df)
BVpt2mm_wetdays = len(BVPpt2mm_df)
BV1mm_wetdays = len(BVP1mm_df)

WH0mm_wetdays = len(WHP0mm_df)
WHpt1mm_wetdays = len(WHPpt1mm_df)
WHleqpt1mm_wetdays = len(WHPleqpt1mm_df)
WHpt2mm_wetdays = len(WHPpt2mm_df)
WH1mm_wetdays = len(WHP1mm_df)

### Creata a dataframe of wet day numbers

In [9]:
# step 1: initialise my lists
wetdays = [["FR_2325m", FR0mm_wetdays, FRpt1mm_wetdays, FRleqpt1mm_wetdays, FRpt2mm_wetdays, FR1mm_wetdays],
           ["UC_1845m", UC0mm_wetdays, UCpt1mm_wetdays, UCleqpt1mm_wetdays, UCpt2mm_wetdays, UC1mm_wetdays],
           ["KP_1610m", KP0mm_wetdays, KPpt1mm_wetdays, KPleqpt1mm_wetdays, KPpt2mm_wetdays, KP1mm_wetdays],
           ["HM_1436m", HM0mm_wetdays, HMpt1mm_wetdays, HMleqpt1mm_wetdays, HMpt2mm_wetdays, HM1mm_wetdays],
           ["BCS_1397m", BCS0mm_wetdays, BCSpt1mm_wetdays, BCSleqpt1mm_wetdays, BCSpt2mm_wetdays, BCS1mm_wetdays],
           ["K_1391m", K0mm_wetdays, Kpt1mm_wetdays, Kleqpt1mm_wetdays, Kpt2mm_wetdays, K1mm_wetdays],
           ["BV_1298m", BV0mm_wetdays, BVpt1mm_wetdays, BVleqpt1mm_wetdays, BVpt2mm_wetdays, BV1mm_wetdays],
           ["WH_1268m", WH0mm_wetdays, WHpt1mm_wetdays, WHleqpt1mm_wetdays, WHpt2mm_wetdays, WH1mm_wetdays]]
wetdays

[['FR_2325m', 2330, 2243, 2243, 2029, 1482],
 ['UC_1845m', 2138, 1925, 1925, 1696, 1129],
 ['KP_1610m', 786, 786, 770, 770, 653],
 ['HM_1436m', 1707, 1551, 1551, 1386, 916],
 ['BCS_1397m', 1808, 1808, 1808, 1808, 980],
 ['K_1391m', 1298, 1298, 1297, 1297, 1027],
 ['BV_1298m', 1543, 1543, 1543, 1543, 947],
 ['WH_1268m', 1195, 1195, 1195, 1195, 919]]

In [36]:
# step 2: create df from lists
wetdays_df = pd.DataFrame(wetdays, columns = ["station", "wetdays_0mm", "wetdays_<0.1mm", "wetdays_<=0.1mm",
                                              "wetdays_<0.2mm", "wetdays<1mm"])
wetdays_df.set_index("station", inplace=True)
wetdays_df

,wetdays_0mm,wetdays_<0.1mm,wetdays_<=0.1mm,wetdays_<0.2mm,wetdays<1mm
station,,,,,
FR_2325m,2330,2243,2243,2029,1482
UC_1845m,2138,1925,1925,1696,1129
KP_1610m,786,786,770,770,653
HM_1436m,1707,1551,1551,1386,916
BCS_1397m,1808,1808,1808,1808,980
K_1391m,1298,1298,1297,1297,1027
BV_1298m,1543,1543,1543,1543,947
WH_1268m,1195,1195,1195,1195,919


### Write dataframe to csv

In [32]:
wetdays_df.to_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Random_mixing\RMWSPy_Horning_and_Haese_2021\RMWSPy-master\MCRB_examples\MCRB_gauges_only\Characterising_P_statistics\no_of_wetdays_allthreshs.csv")

### Calculate the mm difference and %age difference in wet days for < 0.1 mm and < 1.0 mm

In [37]:
# first, drop all cols except those I want to keep.
wetdays_df.drop(["wetdays_0mm", "wetdays_<=0.1mm", "wetdays_<0.2mm"], axis=1, inplace=True)
wetdays_df

,wetdays_<0.1mm,wetdays<1mm
station,,
FR_2325m,2243,1482
UC_1845m,1925,1129
KP_1610m,786,653
HM_1436m,1551,916
BCS_1397m,1808,980
K_1391m,1298,1027
BV_1298m,1543,947
WH_1268m,1195,919


In [38]:
# now calculate differences
# diff in mm
wetdays_df['diff_mm'] = wetdays_df['wetdays_<0.1mm'] - wetdays_df['wetdays<1mm']
# %age diff
wetdays_df['%_diff'] = (wetdays_df['diff_mm'] / wetdays_df['wetdays_<0.1mm']) * 100
wetdays_df

,wetdays_<0.1mm,wetdays<1mm,diff_mm,%_diff
station,,,,
FR_2325m,2243,1482,761,33.927775
UC_1845m,1925,1129,796,41.350649
KP_1610m,786,653,133,16.921120
HM_1436m,1551,916,635,40.941328
BCS_1397m,1808,980,828,45.796460
K_1391m,1298,1027,271,20.878274
BV_1298m,1543,947,596,38.626053
WH_1268m,1195,919,276,23.096234


### Write dataframe to csv

In [40]:
wetdays_df.to_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Random_mixing\RMWSPy_Horning_and_Haese_2021\RMWSPy-master\MCRB_examples\MCRB_gauges_only\Characterising_P_statistics\no_of_wetdays_diff.csv")

In [79]:
# check how many wet days there are at each station (from highest elev to lowest)
print("FR wet days (2325 m) =", len(FRP_df))
print("UC wet days (1845 m) =", len(UCP_df))
print("KP wet days (1610 m) =", len(KPP_df))
print("HM wet days (1436 m) =", len(HMP_df))
print("BCS wet days (1397 m) =", len(BCSP_df))
print("K wet days (1391 m) =", len(KP_df))
print("BV wet days (1298 m) =", len(BVP_df))
print("WH wet days (1268 m) =", len(WHP_df))

FR wet days (2325 m) = 4019
UC wet days (1845 m) = 4019
KP wet days (1610 m) = 4019
HM wet days (1436 m) = 4019
BCS wet days (1397 m) = 4019
K wet days (1391 m) = 4019
BV wet days (1298 m) = 4019
WH wet days (1268 m) = 4019
